# Import

These requirements are necessary if you launch this notebook from SageMaker instances

In [1]:
"""!pip install mlflow
!pip install pytorch-lightning
!pip install transformers
!pip install tqdm
!pip install sagemaker
!pip install s3fs
!pip install smdebug"""

In [2]:
import sys
sys.path.append('../../../')

import os
import sys
import logging
import argparse
from pathlib import Path
from ast import literal_eval
from collections import Counter
from typing import Any, Dict, Optional

In [3]:
from tqdm.auto import tqdm

import torchmetrics
from torchmetrics.functional import accuracy, f1, auroc

import sagemaker
from sagemaker import get_execution_role
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.core.decorators import auto_move_data
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import MLFlowLogger


import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler

import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, multilabel_confusion_matrix


import matplotlib.pyplot as plt
from pylab import rcParams
from matplotlib import rc

import transformers
from transformers import (
    AdamW,
    AutoConfig,
    AutoModel,
    AutoTokenizer,
)
from transformers.optimization import (
    Adafactor,
    get_linear_schedule_with_warmup,
)

Local constants, regarding the data, MLFlow server, paths, etc..: use them

In [4]:
from deep.constants import *
from deep.utils import *

/home/selim/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [5]:
%load_ext autoreload
%autoreload 2

# Data

You can use the data you want. We advise the `pandas` format.

In [6]:
def restrain_df(df):
    return df[df.apply(lambda x: len(x.subpillars_1d)>2 or len(x.subpillars_2d)>2, axis=1)]

In [7]:
TRAIN_PATH = os.path.join('..', '..', '..', "data", "frameworks_data", "data_v0.5.1", "data_v0.5.1_train.csv")
VAL_PATH = os.path.join('..', '..', '..', "data", "frameworks_data", "data_v0.5.1", "data_v0.5.1_val.csv")
TEST_PATH = os.path.join('..', '..', '..', "data", "frameworks_data", "data_v0.5.1", "data_v0.5.1_test.csv")

train_df = restrain_df(pd.read_csv(TRAIN_PATH))
val_df = restrain_df(pd.concat([pd.read_csv(TEST_PATH), pd.read_csv(VAL_PATH)]))

## Sagemaker Prep

### Session

Configure SageMaker

In [8]:
sess = sagemaker.Session(default_bucket=DEV_BUCKET.name)
role = SAGEMAKER_ROLE
role_arn = SAGEMAKER_ROLE_ARN
tracking_uri = MLFLOW_SERVER

### Bucket upload

You need to upload data to an S3 bucket. 




In [9]:
MLFLOW_SERVER

'http://mlflow-deep-387470f3-1883319727.us-east-1.elb.amazonaws.com/'

In [10]:
sample = True  # To make the computations faster, sample = True.

if sample:
    train_df = train_df.sample(n=1000)
    val_df = val_df.sample(n=1000)
    
job_name = f"pytorch-{formatted_time()}-subpillars-model-test-mlflow"  # change it as you prefer
input_path = DEV_BUCKET / 'training' / 'input_data' / job_name  # Do not change this

train_path = str(input_path / 'train.pickle')
val_path = str(input_path / 'val.pickle')


train_df.to_pickle(train_path, protocol=4)  # protocol 4 is necessary, since SageMaker uses python 3.6
val_df.to_pickle(val_path, protocol=4)

### Estimator Definition

In [11]:
# GPU instances

instances = [
    'ml.p2.xlarge',
    'ml.p3.2xlarge'
]

The hyperparameters are passed as command line arguments to the training script. 

You can add/change them as you like. It's important to keep the `tracking_uri` and the `experiment_name` which are used by MLFlow.

The class `PyTorch` is part of the `SageMaker` python API. The parameters are important and you should probably not change most of them. The ones you may want to change are:

- `instance_type`, specify the instance you want
- `source_dir`, specify your script directory. Try to use global variable as much as possible

In [12]:
from sagemaker.pytorch import PyTorch


hyperparameters={
    'tracking_uri': MLFLOW_SERVER,
    'experiment_name': "pl-all-tags",
    'max_len': 512,
    'epochs': 1,
    'model_name': 'microsoft/xtremedistil-l6-h384-uncased',
    'tokenizer_name': 'microsoft/xtremedistil-l6-h384-uncased',
    'dropout_rate': 0.3,
    'pred_threshold':0.4,
    'output_length': 384,
    'learning_rate': 7e-5,
    'training_names':'sectors'
}

estimator = PyTorch(
    entry_point='train_mlflow.py',
    source_dir=str('../../../scripts/training/selim/multiclass-lightning'),
    output_path=str(DEV_BUCKET/'models/'),
    code_location=str(input_path),
    instance_type='ml.p3.2xlarge',
    instance_count=1,
    role=role,
    framework_version='1.8',
    py_version='py36',
    hyperparameters = hyperparameters,
    job_name=job_name,
#     train_instance_count=2,
#     train_instance_type="ml.c4.xlarge",
)

In [13]:
fit_arguments = {
    'train': str(input_path),
    'test': str(input_path)
}

In [14]:
# Fit the estimator

estimator.fit(fit_arguments, job_name=job_name)

2021-09-01 09:18:53 Starting - Starting the training job...
2021-09-01 09:19:16 Starting - Launching requested ML instancesProfilerReport-1630487930: InProgress
......
2021-09-01 09:20:25 Starting - Preparing the instances for training.........
2021-09-01 09:21:57 Downloading - Downloading input data...
2021-09-01 09:22:37 Training - Downloading the training image.....................bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2021-09-01 09:26:27,208 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2021-09-01 09:26:27,233 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2021-09-01 09:26:27,242 sagemaker_pytorch_container.training INFO     Invoking user training script.
2021-09-01 09:26:27,694 sagemaker-training-toolkit INFO     Installing dependencies from requirements.txt:
/opt/conda/bin/python3.6 -m pip install -r requirements.txt


  Created wheel for nltk: filename=nltk-3.2.5-py3-none-any.whl size=1392144 sha256=07993d4f50a93b4e562c88037ea773a720b0fc801e018b6ff7d5c4ab711a6de3
  Stored in directory: /root/.cache/pip/wheels/f2/7f/71/cb36468789a03b5e2908281c8e1ce093e6860258b6b61677d8
  Created wheel for sagemaker: filename=sagemaker-2.49.1-py2.py3-none-any.whl size=591917 sha256=b8344df773eaa195f064d6018eb562a65648a3023883d1872fbe38c65dd011e5
  Stored in directory: /root/.cache/pip/wheels/4c/af/ea/8ff5943a87155df5b184e54474fbf2b59b75e5c172854643c6
  Created wheel for aiobotocore: filename=aiobotocore-1.4.1-py3-none-any.whl size=49822 sha256=bb983a10de7fdb3be60e3d7f84855ab9ed4fced04a52849d983a1292b76c04ce
  Stored in directory: /root/.cache/pip/wheels/0e/ee/c0/a79457b0bdf63d19a62fb58df7d8f760e5e4811ef2f7058436
  Created wheel for alembic: filename=alembic-1.4.1-py2.py3-none-any.whl size=158155 sha256=67fe988ea90dabd7c6b234040057eaa3f33a39210cbcf2fb616d927f653d92a5
  Stored in directory: /root/.cache/pip/wheels/e9/7b

  Attempting uninstall: botocore
    Found existing installation: botocore 1.20.110
    Uninstalling botocore-1.20.110:
      Successfully uninstalled botocore-1.20.110
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.51.0
    Uninstalling tqdm-4.51.0:
      Successfully uninstalled tqdm-4.51.0
  Attempting uninstall: h5py
    Found existing installation: h5py 2.8.0
    Uninstalling h5py-2.8.0:
      Successfully uninstalled h5py-2.8.0
  Attempting uninstall: boto3
    Found existing installation: boto3 1.17.110
    Uninstalling boto3-1.17.110:
      Successfully uninstalled boto3-1.17.110

2021-09-01 09:27:38 Training - Training image download completed. Training in progress.  Attempting uninstall: smdebug
    Found existing installation: smdebug 1.0.9
    Uninstalling smdebug-1.0.9:
      Successfully uninstalled smdebug-1.0.9
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.24.2
    Uninstalling scikit-learn-0.24.2:
      Succe

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package omw to /root/nltk_data...
[nltk_data]   Unzipping corpora/omw.zip.
#015Validation sanity check: 0it [00:00, ?it/s]#015Validation sanity check:   0%|          | 0/2 [00:00<?, ?it/s][2021-09-01 09:28:06.670 algo-1:91 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2021-09-01 09:28:06.705 algo-1:91 INFO profiler_config_parser.py:102] User has disabled profiler.
[2021-09-01 09:28:06.706 algo-1:91 INFO json_config.py:91] Creating hook from json_config at /opt/ml/input/config/debughookconfig.json.
[2021-09-01 09:28:06.707 algo-1:91 INFO hook.py:201] tensorboard_dir has not been set for the hook. SMDebug will not be exporting tensorboard summaries.
[2021-09-01 09:28:06.707 algo-1:91 INFO hoo

2021-09-01 09:27:49.269436: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
INFO:root:reading, preprocessing data
2021/09/01 09:27:51 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of pytorch. If you encounter errors during autologging, try upgrading / downgrading pytorch to a supported version, or try upgrading MLflow.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
INFO:filelock:Lock 140414464556000 acquired on /root/.cache/huggingface/transformers/31d6577412393ebb07c02de876b2d1397fcae2d85cb053b588145f6869ab1a15.44cd178af39e607af310bc4cc48a944f5e5f746b372c161b32511f0fd585789b.lock
#015Downloading:   0%|          | 0.00/526 [00:00<?, ?B/s]#015Downloading: 100%|██████████| 526/526 [00:00<00:00, 733kB/s]
INFO:filelock:Lock 140414464556000 released on /root/.cache/huggingface/transformers/31d6577412393ebb07c02de87


2021-09-01 09:28:38 Uploading - Uploading generated training model
2021-09-01 09:28:59 Failed - Training job failed


UnexpectedStatusException: Error for Training job pytorch-2021-09-01-11-18-42-839-subpillars-model-test-mlflow: Failed. Reason: AlgorithmError: ExecuteUserScriptError:
Command "/opt/conda/bin/python3.6 train_mlflow.py --dropout_rate 0.3 --epochs 1 --experiment_name pl-all-tags --learning_rate 7e-05 --max_len 512 --model_name microsoft/xtremedistil-l6-h384-uncased --output_length 384 --pred_threshold 0.4 --tokenizer_name microsoft/xtremedistil-l6-h384-uncased --tracking_uri http://mlflow-deep-387470f3-1883319727.us-east-1.elb.amazonaws.com/ --training_names sectors"
2021-09-01 09:27:49.269436: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
INFO:root:reading, preprocessing data
2021/09/01 09:27:51 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of pytorch. If you encounter errors during autologging, try upgrading / downgrading pytorch to a supported version, or try upgrading MLflow.
INFO:n